In [1]:
import pandas as pd
import os
import shutil
import random
import numpy as np
import math

In [2]:
# Se cagan los tres archivos csv creados con el scipt anterior y se eliminan los registros duplicados
# Se ha observado que muchos de ellos se repiten y el csv de test repite todos
infoAudios = pd.concat([pd.read_csv("../csv/archivoTrain.csv", header=0), pd.read_csv("../csv/archivoValidated.csv", header=0), pd.read_csv("../csv/archivoTest.csv", header=0)]).drop_duplicates()

# Se definen las rutas de las que se extraen los audios y donde se van a almacenar los audios escogidos finalmente para usar con el modelo
# Se ha observado que todos los audios en la carpeta de test ya se encuentran en la de entrenamiento, por lo que no se utilizará esa carpeta
rutaOrigen = "../audios/audiosEntrenamiento"
rutaDestinoEntrenamiento = "../audios/seleccionEntrenamiento"
rutaDestinoRefuerzo = "../audios/seleccionRefuerzo"

In [3]:
# Condicion para que un cliente sea aceptado para entrenar el modelo
cantidadAudiosPorCliente = 5

In [4]:
def obtenerListaFinal(cantidadAudiosPorCliente, topeHombres, dataframe):
    # Diccionario con los id de los diferentes clientes
    idDict = {}
    # Lista con los id de los clientes cuyos audios se utilizaran para entrenar el modelo
    clientesValidos = []
    # Lista con los id de los clientes cuyos audios se utilizaran para refuerzo
    clientesRef = []

    cantidadHombresTrain = 0
    cantidadMujeresTrain = 0
    cantidadHombresCortada = 0
    cantidadHombresCortadaRef = 0
    cantidadHombresRef = 0
    cantidadMujeresRef = 0

    # Por si quiero coger todos los hombres
    if topeHombres == 0:
        topeHombres = len(dataframe)

    # Se cogen los diferentes id del dataframe
    for ids in dataframe['client_id']:
        if ids not in idDict:
            idDict[ids] = 1
        else:
            idDict[ids] += 1

    # Se recorren los id y para cada id se buscan sus audios
    for client in idDict:
        if idDict[client] >= cantidadAudiosPorCliente: # Si tiene mas de los especificados se añade a entrenamiento   >= minAudios and idDict[client] <= maxAudios:   #
            
            if dataframe.loc[dataframe['client_id'] == client]['gender'].iloc[0] == 'male':
                cantidadHombresTrain+=1
                if cantidadHombresTrain <= topeHombres: # se acorta para igualar mas la cantidad de hombres y mujeres
                    clientesValidos.append((client, dataframe.loc[dataframe['client_id'] == client]))
                    cantidadHombresCortada+=1
            else:
                cantidadMujeresTrain+=1
                clientesValidos.append((client, dataframe.loc[dataframe['client_id'] == client]))

        else: # Si no se añade a refuerzo
            if dataframe.loc[dataframe['client_id'] == client]['gender'].iloc[0] == 'male':
                cantidadHombresRef+=1
                if cantidadHombresRef <= 600:
                    clientesRef.append((client, dataframe.loc[dataframe['client_id'] == client]))
                    cantidadHombresCortadaRef+=1
            else:
                cantidadMujeresRef+=1
                clientesRef.append((client, dataframe.loc[dataframe['client_id'] == client]))


    print(f'Se han escogido clientes con minimo {cantidadAudiosPorCliente} audios.\n'+
        f'De {len(idDict)} clientes posibles: \n'+
        f'- {cantidadHombresTrain} ({cantidadHombresCortada} seleccionados) hombres y {cantidadMujeresTrain} mujeres se utilizan para entrenar.\n'+
        f'- {cantidadHombresRef} ({cantidadHombresCortadaRef} seleccionados) hombres y {cantidadMujeresRef} mujeres son para refuerzo.')

    return clientesValidos, clientesRef

In [9]:
def crearCarpetas(clientesValidos, rutaDestino, rutaOrigen, cantidadAudiosPorCliente, contador):

    for cliente in clientesValidos: # Se recorre la lista proporcionada
        
        if cliente[1]['gender'].iloc[0] == 'male': # Se comprueba si es hombre o mujer, para crear el nombre de la carpeta
            nombreCarpeta = 'cli' + str(contador) + "_H"
        else:
            nombreCarpeta = 'cli' + str(contador) + "_M"

        carpetaCliente = os.path.join(rutaDestino, nombreCarpeta)
        os.mkdir(carpetaCliente) # Se crea la carpeta

        # Si el cliente tiene mas audios de los establecidos se eligen aleatoriamente x audios (el valor de cantidadAudiosPorCliente)
        # Pasa solamente cuando son audios de entrenamiento
        if len(cliente[1]) >= cantidadAudiosPorCliente:
            rnd = []
            while len(rnd) != cantidadAudiosPorCliente:
                rNum = random.randint(0, len(cliente[1])-1)
                if rNum not in rnd:
                    rnd.append(rNum)

            for i in range(len(cliente[1])):
                if i in rnd:
                    shutil.copy(os.path.join(rutaOrigen, cliente[1]['path'].iloc[i]), os.path.join(carpetaCliente, cliente[1]['path'].iloc[i]))
        else: # Si no, sera para refuerzo y se copian todos los audios del cliente
            for archivo in cliente[1]['path']:
                shutil.copy(os.path.join(rutaOrigen, archivo), os.path.join(carpetaCliente, archivo))

        print(contador,end="\r")
        contador += 1
    
    print("Carpetas creadas")
    return contador

In [10]:
clientesValidosEntrenamiento, clientesValidosRef = obtenerListaFinal(cantidadAudiosPorCliente, 900, infoAudios)
crearCarpetas(clientesValidosEntrenamiento, rutaDestinoEntrenamiento, rutaOrigen, cantidadAudiosPorCliente, 1)
crearCarpetas(clientesValidosRef, rutaDestinoRefuerzo, rutaOrigen, cantidadAudiosPorCliente, 1)

Se han escogido clientes con minimo 5 audios.
De 4172 clientes posibles: 
- 1637 (900 seleccionados) hombres y 694 mujeres se utilizan para entrenar.
- 1377 (600 seleccionados) hombres y 464 mujeres son para refuerzo.
Carpetas creadas
Carpetas creadas


1065